# Evaluation for the third model

In [1]:
from pathlib import Path
import numpy as np
import torch 

In [2]:
# Import local modules from 'src/utils' as package 'utils'
import sys; sys.path.insert(0, '../')

In [3]:
from utils.models.transformer import TransformerEncoderOnly, get_loss_function

## Read Datasets from .csv

In [4]:
from utils.file_io import read_trajectory_datasets

In [5]:
data_folder = Path("../../data/")
train_set, validation_set, test_set, visualization_set = read_trajectory_datasets(data_folder, 0.8, 0.15, 0.045, 0.005, 64, standardize_features=True)

TypeError: read_trajectory_datasets() got an unexpected keyword argument 'standardize_features'

In [ ]:
# FIXME: Total loaded size correct?
input_shape, output_shape = 8, 3
print(f"Data shape {input_shape} / {output_shape} of total {len(visualization_set)} data rows!")

## Load parameter, functions and dataloader

In [ ]:
from utils.file_io import define_dataloader_from_angle_dataset

In [ ]:
model_path = Path("../../models/transformer/").absolute()

## Load trail grid and best train loss

In [ ]:
import pandas as pd
import ast

In [ ]:
result_grid = pd.read_csv(model_path / "trail_grid_.csv")
result_grid.sort_values('loss').head(5)

In [ ]:
# Load best train loss
best_result = result_grid.iloc[result_grid['loss'].idxmin()].to_dict()
trail_id = best_result['trial_id']

print(f"Trail ID from the best run: {trail_id}")

## 1. Analysis: Compute test loss

In [ ]:
from utils.file_io import define_dataloader_from_subset
from utils.evaluation import compute_sliding_window_predictions, compute_losses_from
from utils.loss_functions import maximum_squared_error

In [ ]:
_, _, test_dataloader = define_dataloader_from_subset(train_set, validation_set, test_set, batch_size=best_result['config/batch_size'], shuffle=True)

In [ ]:
 model = TransformerEncoderOnly(input_shape, best_result["config/model_dim"], best_result["config/feedforward_dim"], output_shape, best_result["config/encoder_layer"],
                                best_result["config/transformer_dropout"], best_result["config/pos_encoder_dropout"])
model.load_state_dict(best_model)
model.eval()

In [ ]:
y, y_true = compute_sliding_window_predictions(test_dataloader, model, 'cpu')
test_losses = compute_losses_from(y, y_true, get_loss_function())
print(f"The mean squared error of the loaded model on test is: {test_losses.mean()}")

## 2. Analysis: Trace animation

In [ ]:
from utils.visualization import create_trace_animation
from matplotlib import pyplot as plt
from IPython.display import HTML

In [ ]:
%matplotlib notebook
 
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  

In [ ]:
animation = create_trace_animation(y.numpy(), y_true.numpy())
HTML(animation.to_jshtml())